<a id='top'></a>
# Spec Mode Data Calibration: Pipeline Stage 2
---
**Authors**:  
Maria Pena-Guerrero (pena@stsci.edu), Jo Taylor (jotaylor@stsci.edu)

**Latest Update**: 19 May 2021

<div class="alert alert-block alert-warning">
    <h3><u><b>Notebook Goals</b></u></h3>
    <ul>Working with the Spectroscopic Stage 2 Calibration Pipeline, we will:</ul>    
<ul>
    <li>Look at the different ways to call the pipeline</li>
    <li>Examine exactly what each pipeline step does to the science data</li>    
</ul>
</div>

## Table of Contents
* [Introduction](#intro)
* [Pipeline Resources and Documentation](#resources)
   * [Installation](#installation)
   * [Reference Files](#reference_files)
* [Imports](#imports)
* [Convenience Functions](#convenience_functions)
* [Download Data](#download_data)
* [Methods for calling steps/pipelines](#calling_methods)
* [Parameter reference files](#parameter_reffiles)
* [calwebb_spec2 - count rate files](#spec2) 
   * [Run the entire pipeline](#spec2_at_once)
       * [run() method](#run_method)
       * [call() method](#call_method)
       * [command line](#command_line)
   * [Run the individual pipeline steps](#spec2_step_by_step)
       * [The `Assign WCS` step](#assign_wcs)
       * [The `Background Subtraction` step](#background)
       * [The `Imprint Subtraction` step](#imprint)
       * [The `MSA Flagging` step](#msaflagging)
       * [The `Extract_2d` step](#extract_2d)
       * [The `Source Type` step](#srctype)
       * [The `Master Background` step](#masterbg)
       * [The `Wavecorr` step](#wavecorr)
       * [The `Flat Field` step](#flat_field)
       * [The `Straylight` step](#straylight)
       * [The `Fringe` step](#fringe)
       * [The `Pathloss` step](#pathloss)
       * [The `Barshadow` step](#barshadow)
       * [The `Photom` step](#photom)
       * [The `Resample Spec` step](#resample)
       * [The `Cube Build` step](#cube_build)
       * [The `Extract_1d` step](#extract_1d)


<a id='intro'></a>
## Introduction

This notebook covers the Stage 2 processing of the spectroscopic mode data calibration module. In this notebook we will review how to run Stage 2 of the JWST calibration pipeline, also known as *calwebb_spec2*. 

The [Stage 2 pipeline](https://jwst-pipeline.readthedocs.io/en/stable/jwst/pipeline/calwebb_spec2.html#calwebb-spec2) applies corrections to countrate files, which are the output files of the Stage 1 pipeline that works for all exposure types (imaging, spectroscopic, coronagraphic, etc.). The Stage 1 pipeline is applied to one exposure at a time, beginning with an uncalibrated multiaccum ramp (*_uncal.fits file*). It is sometimes referred to as “ramps-to-slopes” processing. For additional information, please see the [Stage 1 pipeline](https://jwst-pipeline.readthedocs.io/en/stable/jwst/pipeline/calwebb_detector1.html#calwebb-detector1) documentation and/or the corresponding notebooks.

Each input file of *calwebb_spec2* is composed of one (*_rate.fits file*) or more (*_rateints.fits file*) exposures. For details on pipeline products, see [Pipeline Stages](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/main.html).

The output is a fully calibrated individual exposure, with units of MJy/str (or MJy for NIRSpec point sources).

The *calwebb_spec2* pipeline of Stage 2, is the “Swiss army knife” of pipeline modules. It contains corrections that are only applied to certain instruments and/or instrument modes. 

<a id='resources'></a>
## Pipeline Resources and Documentation

There are several different places to find information on installing and running the pipeline. This notebook will summarize each Stage 2 spectroscopic pipeline step. To find more in-depth instructions use the links below.

* [JWST Documentation (JDox) for the Stage 2 spectroscopic pipeline](https://jwst-docs.stsci.edu/jwst-data-reduction-pipeline/stages-of-processing/calwebb_spec2) including short a short summary of what each step does.

* [High-level description of all pipeline stages and steps](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/main.html) from the pipeline software documentation website.

* [`jwst` package documentation](https://jwst-pipeline.readthedocs.io/en/latest/jwst/introduction.html) including how to run the pipeline, input/output files, etc.

* [`jwst` package GitHub repository, with installation instructions](https://github.com/spacetelescope/jwst/blob/master/README.md)

* [**Help Desk**](https://stsci.service-now.com/jwst?id=sc_cat_item&sys_id=27a8af2fdbf2220033b55dd5ce9619cd&sysparm_category=e15706fc0a0a0aa7007fc21e1ab70c2f): **If you have any questions or problems regarding the pipeline, submit a ticket to the Help Desk**

<a id='installation'></a>
### Installation

<div class="alert alert-block alert-info">
    During the JWebbinar, we will be working in a pre-existing environment where the <b>jwst</b> package has already been installed, so you won't need to install it yourself.
</div>

<div class="alert alert-block alert-warning">
If you wish to run this notebook outside of this JWebbinar, you will have to first install the <b>jwst</b> package.<br>

For more detailed instructions on the various ways to install the package, see the [installation instructions](https://github.com/spacetelescope/jwst/blob/master/README.md) on GitHub.

The easiest way to install the pipeline is via `pip`. Below we show how to create a new conda environment, activate that environment, and then install the latest released version of the pipeline. You can name your environment anything you like. In the lines below, replace `<env_name>` with your chosen environment name.

>`conda create -n <env_name> python`<br>
>`conda activate <env_name>`<br>
>`pip install jwst`

If you wish to install the development version of the pipeline, which is more recent than (but not as well tested compared to) the latest released version:

>`conda create -n <env_name> python`<br>
>`conda activate <env_name>`<br>
>`pip install git+https://github.com/spacetelescope/jwst`
    
</div>

<a id='reference_files'></a>
### Reference Files

[Calibration reference files](https://jwst-docs.stsci.edu/data-processing-and-calibration-files/calibration-reference-files) are a collection of FITS and ASDF files that are used to remove instrumental signatures and calibrate JWST data. For example, the dark current reference file contains a multiaccum ramp of dark current signal to be subtracted from the data during the dark current subtraction step. 

When running a pipeline or pipeline step, the pipeline will automatically look for any required reference files in a pre-defined local directory. If the required reference files are not present, they will automatically be downloaded from the Calibration Reference Data System (CRDS) at STScI.

<div class="alert alert-block alert-info">
    During the JWebbinar, our pre-existing existing environment is set up to correctly use and store calibration reference files, and you do not need to set the environment variables below.
</div>
    
<div class="alert alert-block alert-warning">
If you wish to run this notebook outside of this JWebbinar, you will have to specify a local directory in which to store reference files, along with the server to use to download the reference files from CRDS. To accomplish this, there are two environment variables that should be set prior to calling the pipeline. These are the CRDS_PATH and CRDS_SERVER_URL variables. In the example below, reference files will be downloaded to the "crds_cache" directory under the home directory.

>`$ export CRDS_PATH=$HOME/crds_cache`<br>
>`$ export CRDS_SERVER_URL=https://jwst-crds.stsci.edu`<br>
OR:<br>
`os.environ["CRDS_PATH"] = "/user/myself/crds_cache"`<br>
`os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"`<br>

The first time you run the pipeline, the CRDS server should download all of the context and reference files that are needed for that pipeline run, and dump them into the CRDS_PATH directory. Subsequent executions of the pipeline will first look to see if it has what it needs in CRDS_PATH and anything it doesn't have will be downloaded from the STScI cache. 
</div>

<div class="alert alert-block alert-warning">NOTE: Users at STScI should automatically have access to the Calibration Reference Data System (CRDS) cache for running the pipeline, and can skip setting these environment variables.</div>

[Top of Notebook](#top)

<a id='imports'></a>
## Imports

Import packages necessary for this notebook

In [1]:
# Packages that allow us to get information about objects:
import asdf
import copy
import os
import shutil

# Numpy library:
import numpy as np

# For downloading data
import requests

# Astropy tools:
from astropy.io import fits
from astropy.utils.data import download_file
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch

Set up matplotlib for plotting

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl

# Use this version for non-interactive plots (easier scrolling of the notebook)
%matplotlib inline

# Use this version (outside of Jupyter Lab) if you want interactive plots
#%matplotlib notebook

# These gymnastics are needed to make the sizes of the figures
# be the same in both the inline and notebook versions
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}

mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80

Import JWST pipeline-related modules

In [35]:
# List of possible data quality flags
from jwst.datamodels import dqflags

# The entire calwebb_spec2 pipeline
from jwst.pipeline.calwebb_spec2 import Spec2Pipeline

# Individual steps that make up calwebb_spec2 and datamodels
from jwst import datamodels
from gwcs import wcstools
from jwst.assign_wcs.assign_wcs_step import AssignWcsStep
from jwst.background.background_step import BackgroundStep
from jwst.imprint.imprint_step import ImprintStep
from jwst.msaflagopen.msaflagopen_step import MSAFlagOpenStep
from jwst.extract_2d.extract_2d_step import Extract2dStep
from jwst.srctype.srctype_step import SourceTypeStep
from jwst.master_background.master_background_step import MasterBackgroundStep
from jwst.wavecorr.wavecorr_step import WavecorrStep
from jwst.flatfield.flat_field_step import FlatFieldStep
from jwst.straylight.straylight_step import StraylightStep
from jwst.fringe.fringe_step import FringeStep
from jwst.pathloss.pathloss_step import PathLossStep
from jwst.barshadow.barshadow_step import BarShadowStep
from jwst.photom.photom_step import PhotomStep
from jwst.resample import ResampleSpecStep
from jwst.cube_build.cube_build_step import CubeBuildStep
from jwst.extract_1d.extract_1d_step import Extract1dStep

2021-05-21 15:52:50,224 - stpipe - WARNING - /Users/pena/anaconda3/envs/pipedev/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Check which version of the pipeline we are running:

In [4]:
import jwst
print(jwst.__version__)

1.1.1.dev92+ged5a6eae


<a id='convenience_functions'></a>
## Define convenience functions and parameters

Here we define some functions and some parameters that we will use repeatedly throughout the notebook.

In [6]:
# To make everything easier, all files saved by the pipeline
# and pipeline steps will be saved to the working directory.
# This is important for the level 2 and 3 pipelines
# once we are working with association files.
output_dir = './'

In [7]:
# Make sure the output directory exists before downloading any data
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
def download_files(files, output_directory):
    """Given a tuple or list of tuples containing (URL, filename),
    download the given files into the current working directory.
    Downloading is done via astropy's download_file. A symbolic link
    is created in the specified output dirctory that points to the
    downloaded file.
    
    Parameters
    ----------
    files : tuple or list of tuples
        Each 2-tuple should contain (URL, filename), where
        URL is the URL from which to download the file, and
        filename will be the name of the symlink pointing to
        the downloaded file.
        
    output_directory : str
        Name of the directory in which to create the symbolic
        links to the downloaded files
                
    Returns
    -------
    filenames : list
        List of filenames corresponding to the symbolic links
        of the downloaded files
    """
    # In the case of a single input tuple, make it a
    # 1 element list, for consistency.
    filenames = []
    if isinstance(files, tuple):
        files = [files]
        
    for file in files:
        filenames.append(file[1])
        if not os.path.isfile(os.path.join(output_directory, file[1])):
            print('Downloading {}...'.format(file[1]))
            demo_file = download_file(file[0], cache=True)
            # Make a symbolic link using a local name for convenience
            os.symlink(demo_file, os.path.join(output_directory, file[1]))
        else:
            print('{} already exists, skipping download...'.format(file[1]))
            continue
    return filenames

[Top of Notebook](#top)

<a id='download_data'></a>
## Download Data

For this module, we will use an uncalibrated NIRCam simulated imaging exposure that is stored in Box. Let's grab it:

In [ ]:
rate_info = ('https://stsci.box.com/shared/static/j46wpyirlbqo30e7c9719ycnuc1qk2lu.fits',
              'jw98765001001_01101_00003_nrcb5_uncal.fits')
rate_file = download_files(uncal_info, output_dir)[0]

In [ ]:
# Download example parameter reference files
param_info = [('https://stsci.box.com/shared/static/nmaipgvlferrz95eyksra4zmlhx2m5bm.asdf',
               'detector1pipeline_modified_paramfile.asdf'),
              ('https://stsci.box.com/shared/static/kww0o8d77h2u03b38gtdr34g8vqygx8w.asdf',
               'refpix_paramfile.asdf'),
              ('https://stsci.box.com/shared/static/c95paaz9gsvrwwo3slx8k9qav242idi1.asdf',
               'refpix_no_side_pix_paramfile.asdf')]
detector1_param_reffile, refpix_param_reffile, refpix_param_reffile_no_side_pix = \
    download_files(param_info, output_dir)

<a id='calling_methods'></a>
## Methods for calling steps/pipelines

There are three common methods by which the pipeline or pipeline steps can be called. From within python, the `run()` and `call()` methods of the pipeline or step classes can be used. Alternatively, the `strun` command can be used from the command line. Within this notebook, in the section where we [call the entire pipeline](#detector1_at_once), as well as the section where we [call the Reference Pixel Subtraction](#refpix) step, we show examples of all three methods. For the remainder of the pipeline steps, we will focus on using the `run()` method.

When using the `call()` method or `strun`, optional input parameters can be specified via [parameter reference files](#parameter_reffiles). When using the `run()` method, these parameters are instead specified within python.

<a id='parameter_reffiles'></a>
## Parameter Reference Files

When calling a pipeline or pipeline step using the `call()` method or the command line, [parameter reference files](https://jwst-pipeline.readthedocs.io/en/stable/jwst/stpipe/config_asdf.html#config-asdf-files) can be used to specify values for input parameters. These reference files are in [asdf](https://asdf.readthedocs.io/en/stable/) format and appear somewhat similar to json files when examined in a text editor. 

Versions of parameter reference files containing default parameter values for each step and pipeline are available in CRDS. When using the `call()` method, if you do not specify a parameter reference file name in the call, the pipeline or step will retrieve and use the appropriate file from CRDS, which will then run the pipeline or step with the parameter values in that file. If you provide the name of a parameter reference file, then the parameter values in that file will take precedence. For any parameter not specified in your parameter reference file, the pipeline will use the default value.

When using `strun`, the parameter reference file is a required input.

In [ ]:
# Don't forget to close the file


The top part of the file contains various metadata entries about the file itself. Below that, you'll see a `'name'` entry, which lists `Detector1Pipeline` as the class to which these parameters apply. The next line contains the `parameters` entry, which lists parameters and values attached to the pipeline itself. Below this is the `steps` entry, which contains a list of dictionaries. Each dictionary refers to one step within the pipeline, and specifies parameters and values that apply to that step. If you look through these entries, you'll see the same parameters and values that we specified manually when using the `run()` method above.

[Top of Notebook](#top)

---
<a id='spec2'></a>
## The calwebb_spec2 pipeline: count rate files

In the sections below, we will run the Stage 2 pipeline on a single rate XXXX file. We will first call the entire [*calwebb_spec2* pipeline](https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_spec2.html#calwebb-spec2) on the file. The pipeline is a wrapper which will string together all of the appropriate steps in the proper order. 

The final output from this call is an calibrated exposure image which is ready to go into the Stage 3 pipeline. In Stage 2 the flux calibration will be applied, at which point the data will be in physical units (e.g. MJy/str).

See [Figure 1](https://jwst-docs.stsci.edu/jwst-data-reduction-pipeline/stages-of-processing/calwebb_spec2) on the calwebb_spec2 algorithm page for a map of which steps are performed on NIR data and which are used for MIRI data.

In [20]:
input_file_base = os.path.basename(rate_file).replace('rate.fits', '')

In [15]:
rate_file = '/Users/pena/Documents/PyCharmProjects/nirspec/pipeline/testing_data/FS_FULL_FRAME/G140M_opaque/fs_fullframe_g140m_f100lp_NRS1_rate.fits'

<a id='spec2_at_once'></a>
### Run the entire `calwebb_spec2` pipeline

In this section we show how to run the entire calwebb_spec2 pipeline with a single call. In this case the pipeline code can determine which instrument was used to collect the data and runs the appropriate steps in the proper order.

We set parameter values for some of the individual steps, save some outputs, etc, and then call the pipeline.

We will call the pipeline using the `run()` method and while that is running, we will go over the equivalent `call()` and `strun` commands, examine some of the pipeline log entries that are printed to the screen, and then look at the pipeline output.

<a id='run_method'></a>
#### Call the pipeline using the run() method

When using the `run()` method to execute a pipeline (or step), the pipeline class is first instantiated without the data to be processed. Optional input parameters are specified using attributes of the class instance. Finally, the call to the `run()` method is made and the data are supplied.  See here for [more examples of the run() method](https://jwst-pipeline.readthedocs.io/en/stable/jwst/stpipe/call_via_run.html).

The `run()` method does not take any kind of parameter reference file as input. If you wish to set values for various parameters, you must do that manually. Below, we set several parameters in order to show how it's done. 

How do you know what parameters are available to be set and what their default values are? The `spec` property for individual steps will list them. The property is less useful for the pipelines themselves, as it does not show the parameters for the steps comprising the pipeline.

All steps and pipelines have several common parameters that can be set. 

* `save_results` specifies whether or not to save the output of that step/pipeline to a file. The default is False.
* `output_dir` is the directory into which the output files will be saved.
* `output_file` is the base filename to use for the saved result. Note that each step/pipeline will add a custom suffix onto output_file. 

<a id='detector1_using_run'></a>

<div class="alert alert-block alert-info">
Finally, let's run the pipeline. The output can be a little overwhelming. There will be multiple log entries printed to the screen for each step.
</div>

In [16]:
# Create an instance of the pipeline class
step = Spec2Pipeline()

# Set some parameters that pertain to the
# entire pipeline
step.save_results = True
output_dir = '/Users/pena/Documents/NIRSpec_projects/jwebbinars/output_spec_modules'
step.output_dir = output_dir

# Set some parameters that pertain to some of
# the individual steps


# Call the run() method
result = step.run(rate_file)

2021-05-21 14:48:54,413 - stpipe.Spec2Pipeline - INFO - Spec2Pipeline instance created.
2021-05-21 14:48:54,416 - stpipe.Spec2Pipeline.bkg_subtract - INFO - BackgroundStep instance created.
2021-05-21 14:48:54,418 - stpipe.Spec2Pipeline.assign_wcs - INFO - AssignWcsStep instance created.
2021-05-21 14:48:54,419 - stpipe.Spec2Pipeline.imprint_subtract - INFO - ImprintStep instance created.
2021-05-21 14:48:54,420 - stpipe.Spec2Pipeline.msa_flagging - INFO - MSAFlagOpenStep instance created.
2021-05-21 14:48:54,422 - stpipe.Spec2Pipeline.extract_2d - INFO - Extract2dStep instance created.
2021-05-21 14:48:54,426 - stpipe.Spec2Pipeline.master_background - INFO - MasterBackgroundNrsSlitsStep instance created.
2021-05-21 14:48:54,427 - stpipe.Spec2Pipeline.master_background.flat_field - INFO - FlatFieldStep instance created.
2021-05-21 14:48:54,429 - stpipe.Spec2Pipeline.master_background.pathloss - INFO - PathLossStep instance created.
2021-05-21 14:48:54,431 - stpipe.Spec2Pipeline.master_

2021-05-21 14:48:55,078 - stpipe.Spec2Pipeline - WARNING - /Users/pena/anaconda3/envs/pipedev/lib/python3.9/site-packages/stdatamodels/validate.py:34: ValidationWarning: While validating meta.wcsinfo.spectral_order the following error occurred:
1.0 is not of type 'integer'

Failed validating 'type' in schema:
    OrderedDict([('title', 'Spectral order number'),
                 ('type', 'integer'),
                 ('fits_keyword', 'SPORDER'),
                 ('fits_hdu', 'SCI'),
                 ('blend_table', True)])

On instance:
    1.0
  warnings.warn(errmsg, ValidationWarning)

2021-05-21 14:48:55,249 - stpipe.Spec2Pipeline - INFO - Prefetching reference files for dataset: 'fs_fullframe_g140m_f100lp_NRS1_rate.fits' reftypes = ['apcorr', 'area', 'barshadow', 'camera', 'collimator', 'cubepar', 'dflat', 'disperser', 'distortion', 'drizpars', 'extract1d', 'fflat', 'filteroffset', 'flat', 'fore', 'fpa', 'fringe', 'ifufore', 'ifupost', 'ifuslicer', 'msa', 'msaoper', 'ote', 'pathloss'

2021-05-21 14:48:58,154 - stpipe.Spec2Pipeline.assign_wcs - INFO - gwa_ytilt is 0.1260581910610199 deg
2021-05-21 14:48:58,155 - stpipe.Spec2Pipeline.assign_wcs - INFO - gwa_xtilt is 0.3316612243652344 deg
2021-05-21 14:48:58,156 - stpipe.Spec2Pipeline.assign_wcs - INFO - theta_y correction: 0.0002496099796717191 deg
2021-05-21 14:48:58,157 - stpipe.Spec2Pipeline.assign_wcs - INFO - theta_x correction: 0.0 deg
2021-05-21 14:48:58,711 - stpipe.Spec2Pipeline.assign_wcs - INFO - Removing slit S200B1 from the list of open slits because the WCS bounding_box is completely outside the detector.
2021-05-21 14:48:58,712 - stpipe.Spec2Pipeline.assign_wcs - INFO - Slits projected on detector NRS1: ['S200A1', 'S200A2', 'S400A1', 'S1600A1']
2021-05-21 14:48:58,713 - stpipe.Spec2Pipeline.assign_wcs - INFO - Computing WCS for 4 open slitlets
2021-05-21 14:48:58,732 - stpipe.Spec2Pipeline.assign_wcs - INFO - gwa_ytilt is 0.1260581910610199 deg
2021-05-21 14:48:58,733 - stpipe.Spec2Pipeline.assign_wcs 

2021-05-21 14:49:00,952 - stpipe.Spec2Pipeline.extract_2d - INFO - Name of subarray extracted: S400A1
2021-05-21 14:49:00,953 - stpipe.Spec2Pipeline.extract_2d - INFO - Subarray x-extents are: 596 2048
2021-05-21 14:49:00,953 - stpipe.Spec2Pipeline.extract_2d - INFO - Subarray y-extents are: 976 1030
2021-05-21 14:49:01,163 - stpipe.Spec2Pipeline.extract_2d - INFO - set slit_attributes completed
2021-05-21 14:49:01,176 - stpipe.Spec2Pipeline.extract_2d - INFO - Update S_REGION to POLYGON ICRS  156.187759056 -45.682331806 156.186793281 -45.681401531 156.186663358 -45.681468621 156.187629101 -45.682398919
2021-05-21 14:49:01,177 - stpipe.Spec2Pipeline.extract_2d - INFO - Updated S_REGION to POLYGON ICRS  156.187759056 -45.682331806 156.186793281 -45.681401531 156.186663358 -45.681468621 156.187629101 -45.682398919
2021-05-21 14:49:01,408 - stpipe.Spec2Pipeline.extract_2d - INFO - Name of subarray extracted: S1600A1
2021-05-21 14:49:01,408 - stpipe.Spec2Pipeline.extract_2d - INFO - Subarr

2021-05-21 14:49:53,551 - stpipe.Spec2Pipeline.pathloss - INFO - Using PATHLOSS reference file /Users/pena/crds_cache/references/jwst/nirspec/jwst_nirspec_pathloss_0001.fits
2021-05-21 14:49:53,636 - stpipe.Spec2Pipeline.pathloss - INFO - Input exposure type is NRS_FIXEDSLIT
2021-05-21 14:49:54,310 - stpipe.Spec2Pipeline.pathloss - INFO - Working on slit S200A1
2021-05-21 14:49:54,311 - stpipe.Spec2Pipeline.pathloss - INFO - Using aperture S200A1
2021-05-21 14:49:54,338 - stpipe.Spec2Pipeline.pathloss - INFO - Working on slit S200A2
2021-05-21 14:49:54,339 - stpipe.Spec2Pipeline.pathloss - INFO - Using aperture S200A2
2021-05-21 14:49:54,365 - stpipe.Spec2Pipeline.pathloss - INFO - Working on slit S400A1
2021-05-21 14:49:54,366 - stpipe.Spec2Pipeline.pathloss - WARNING - Cannot find matching pathloss model for S400A1
2021-05-21 14:49:54,367 - stpipe.Spec2Pipeline.pathloss - WARNING - Skipping pathloss correction for this slit
2021-05-21 14:49:54,368 - stpipe.Spec2Pipeline.pathloss - WA

2021-05-21 14:49:58,441 - stpipe.Spec2Pipeline.resample_spec - INFO - Drizzling (54, 1452) --> (39, 1452)
2021-05-21 14:49:58,505 - stpipe.Spec2Pipeline.resample_spec - INFO - Update S_REGION to POLYGON ICRS  156.186749042 -45.682353788 156.187682741 -45.682353788 156.187682741 -45.681454385 156.186749042 -45.681454385
2021-05-21 14:49:58,588 - stpipe.Spec2Pipeline.resample_spec - WARNING - /Users/pena/anaconda3/envs/pipedev/lib/python3.9/site-packages/gwcs/geometry.py:203: RuntimeWarning: invalid value encountered in remainder
  lon = np.mod(lon, 360.0 * u.deg if nquant else 360.0)

2021-05-21 14:49:58,855 - stpipe.Spec2Pipeline.resample_spec - INFO - Resampling slit S1600A1 (16, 1428)
2021-05-21 14:49:58,918 - stpipe.Spec2Pipeline.resample_spec - WARNING - /Users/pena/anaconda3/envs/pipedev/lib/python3.9/site-packages/gwcs/geometry.py:203: RuntimeWarning: invalid value encountered in remainder
  lon = np.mod(lon, 360.0 * u.deg if nquant else 360.0)

2021-05-21 14:49:58,930 - stpipe.S

<a id='call_method'></a>
#### Call the pipeline using the call() method

When using the `call()` method, a single command will instantiate and run the pipeline (or step). The input data and optional parameter reference files are supplied in this single command. In this case, any desired input parameters cannot be set after instantiation, as with the `run()` method. See here for [example usage of call() method](https://jwst-pipeline.readthedocs.io/en/stable/jwst/stpipe/call_via_call.html).

The commands below will call the pipeline using the `call()` method and will supply the parameter reference file. Since we just ran the pipeline with the `run()` method above, we won't actually execute the call to `call()`. But if you wish to try it out, use the pull-down menu above to change the cell to be 'Code', and then execute it. (Or, Click 'Cell' > 'Cell Type' > 'Code')

<div class="alert alert-block alert-info">

<b>Method #1:</b>
Provide the name of the observation file, the pipeline-specific input paramters, and the name of the parameter reference file that specifies step-specific parameters
</div>

<div class="alert alert-block alert-info">

<b>Method #2:</b>
In this case, build a nested dictionary that specifies parameter values for various steps, and provide it in the call to call().
</div>

<a id='command_line'></a>
#### Call the pipeline using the command line

Calling a pipeline or step from the command line is similar to using the `call()` method. The data file to be processed, along with an optional parameter reference file and optional parameter/value pairs can be provided to the `strun` command. See here for [additional examples of command line calls](https://jwst-pipeline.readthedocs.io/en/stable/jwst/introduction.html?highlight=%22command%20line%22#running-from-the-command-line).

The cells below contains two different command line commands that use `strun` to call the calwebb_spec2 pipeline. 

<div class="alert alert-block alert-info">

<b>Method #1:</b>
We provide the name of the pipeline class, the observation file, and explicitly set pipeline- and step-specific parameters. You can see that the command quickly becomes quite large with the added parameter settings. 
    
```
    strun jwst.pipeline.Spec2Pipeline jw98765001001_01101_00003_nrsb5_rate.fits --save_results=True --output_dir='./' --steps.flat_field.save_interpolated_flat=True --steps.pathloss.override_pathloss=jwst-nirspec-a400.plrf.fits 
```
</div>

<div class="alert alert-block alert-info">

<b>Method #2:</b>
This version of the command is much more succinct, as the parameter values to be set are all contained within the parameter reference file. The pipeline class is also contained in the parameter reference file, so there is no need to specify it in the command itself.
    
```
    strun spec2_modified_paramfile.asdf jw98765001001_01101_00003_nrsb5_rate.fits
```
</div>

[Top of Notebook](#top)

<a id='detector1_step_by_step'></a>
## Run the individual pipeline steps

In the sections below we show how to run the individual steps and a corresponding brief description.

<a id='assign_wcs'></a>
### The `Assign WCS` step

#### Summary

This step transforms positions in the detector frame to positions in a world coordinate system (WCS) - ICRS and wavelength. There may be intermediate coordinate frames depending on the instrument (e.g. from detector to slit frame). The WCS is saved in the ASDF extension of the FITS file, and it can be accessed as an attribute of the meta object when the fits file is opened as a data model. More information can be found at [assign_wcs](https://jwst-pipeline.readthedocs.io/en/stable/jwst/assign_wcs/index.html).

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/assign_wcs/main.html) of the step.

#### Arguments

There are no optional arguments for this step

#### Reference files used

This step uses various reference files, see [assign_wcs_reference_files](https://jwst-pipeline.readthedocs.io/en/stable/jwst/assign_wcs/reference_files.html). 


#### Run the step

In [26]:
# Using the run() method. Instantiate and set parameters
step = AssignWcsStep()

# Call the run() method on the uncal file
step.save_results = True
step.output_dir = output_dir


# Call the run() method
result = step.run(rate_file)

2021-05-21 15:40:10,637 - stpipe.AssignWcsStep - INFO - AssignWcsStep instance created.
2021-05-21 15:40:11,008 - stpipe.AssignWcsStep - INFO - Step AssignWcsStep running with args ('/Users/pena/Documents/PyCharmProjects/nirspec/pipeline/testing_data/FS_FULL_FRAME/G140M_opaque/fs_fullframe_g140m_f100lp_NRS1_rate.fits',).
2021-05-21 15:40:11,010 - stpipe.AssignWcsStep - INFO - Step AssignWcsStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': '/Users/pena/Documents/NIRSpec_projects/jwebbinars/output_spec_modules', 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'sip_approx': True, 'sip_max_pix_error': 0.25, 'sip_degree': None, 'sip_max_inv_pix_error': 0.25, 'sip_inv_degree': None, 'sip_npoints': 32, 'slit_y_low': -0.55, 'slit_y_high': 0.55}
2021-05-21 15:40:11,113 - stpipe.AssignWcsStep - WARNING - /Users/pena/anaconda3/envs

We have the datamodel instance of the output available already, but if you wanted to open the output file from the data quality initiailzation step, the output file has the same name as the input file, with the original suffix replaced by "assign_wcs".

In [29]:
# Generate the assign_wcs step output file from the rate file name
output_file = os.path.join(output_dir, '{}assignwcsstep.fits'.format(input_file_base))
print(output_file)

/Users/pena/Documents/NIRSpec_projects/jwebbinars/output_spec_modules/fs_fullframe_g140m_f100lp_NRS1_assignwcsstep.fits


In [69]:
# Open the data models
img = datamodels.ImageModel(output_file)
# see everything in the model
#d = img.to_flat_dict()
#for k,v in d.items():
#    print(k, v)

# Print basic information
det = img.meta.instrument.detector
lamp = img.meta.instrument.lamp_state
grat = img.meta.instrument.grating
filt = img.meta.instrument.filter
msg = "from assign_wcs file --> Detector: " + det + "   Grating: " + grat + "   Filter: " + filt + "   Lamp: " + lamp
print(msg)
subarray = img.meta.subarray
print('Subarray: ', subarray)

print("GWA_XTILT: {}".format(img.meta.instrument.gwa_xtilt))
print("GWA_YTILT: {}".format(img.meta.instrument.gwa_ytilt))
print("GWA_TTILT: {}".format(img.meta.instrument.gwa_tilt), "\n")

# To get the open and projected on the detector slits of the pipeline processed file
open_slits = img.meta.wcs.get_transform('gwa', 'slit_frame').slits

for opslit in open_slits:
    pipeslit = opslit.name
            
    print("\nWorking with slit: ", pipeslit)
    
    # get the wcs for this slit
    wcs_slit = jwst.assign_wcs.nirspec.nrs_wcs_set_input(model, pipeslit)
    available_frames = wcs_slit.available_frames
    print("Avalable frames: ", available_frames)


2021-05-21 16:49:22,012 - stpipe - WARNING - /Users/pena/anaconda3/envs/pipedev/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



from assign_wcs file --> Detector: NRS1   Grating: G140M   Filter: F100LP   Lamp: LINE1
Subarray:  <stdatamodels.properties.ObjectNode object at 0x7ffc202af100>
GWA_XTILT: 0.3316612243652344
GWA_YTILT: 0.1260581910610199
GWA_TTILT: 40.28447479156018 


Working with slit:  S200A1
Avalable frames:  ['detector', 'sca', 'gwa', 'slit_frame', 'msa_frame', 'oteip', 'v2v3', 'v2v3vacorr', 'world']

Working with slit:  S200A2
Avalable frames:  ['detector', 'sca', 'gwa', 'slit_frame', 'msa_frame', 'oteip', 'v2v3', 'v2v3vacorr', 'world']

Working with slit:  S400A1
Avalable frames:  ['detector', 'sca', 'gwa', 'slit_frame', 'msa_frame', 'oteip', 'v2v3', 'v2v3vacorr', 'world']

Working with slit:  S1600A1
Avalable frames:  ['detector', 'sca', 'gwa', 'slit_frame', 'msa_frame', 'oteip', 'v2v3', 'v2v3vacorr', 'world']


[Top of Notebook](#top)

<a id='background'></a>
## The `Background` step

#### Summary

The background subtraction step performs image-from-image subtraction, to remove the background signal. The step takes as input one target exposure and one or more background exposures, which are then subtracted from the target exosure.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/background/description.html) of the step.

#### Arguments

There are two optional arguments for this step, that are only used for non-Wide-Field Slitless Spectroscopy (WFSS) Slitless Spectroscopy exposures. Please see the documentation for further detail [optional_arguments](https://jwst-pipeline.readthedocs.io/en/stable/jwst/background/arguments.html).

#### Reference files used

The background image subtraction step uses reference files only when processing WFSS exposures [WFSS_reference_files](https://jwst-pipeline.readthedocs.io/en/stable/jwst/background/reference_files.html).

#### Run the step

In [ ]:
# Using the run() method. Instantiate and set parameters
step = BackgroundStep()

# Call the run() method on the uncal file
step.save_results = True
step.output_dir = output_dir
bkg_list = [background_img1, background_img2]

# Call the run() method
result = step.run(result, bkg_list)

 <a id='imprint'> </a>
## The `Imprint Subtraction` step

#### Summary

This is a NIRSpec-specific step, particularly for the MSA and IFU modes. The step removes patterns created in NIRSpec MOS and IFU exposures by the MSA metal grid of the little shutters. Thes is pattern removal is done by subtracting a dedicated exposure taken with all MSA shutters closed and the IFU entrance aperture blocked.

The Step uses as input the resulting file of the Background subtraction step or the Assign WCS, if there were no
background exposures taken.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/imprint/description.html) of the step.

#### Arguments

There are no optional arguments for this step.

#### Reference files used

This step uses no specific reference files.


#### Run the step

In [ ]:
# Using the run() method. Instantiate and set parameters
step = ImprintStep()

# Call the run() method on the uncal file
step.save_results = True
step.output_dir = output_dir
msa_imprint = '/somewhere/msa_imprint.fits'

# Call the run() method
result = step.run(result, msa_imprint)

[Top of Notebook](#top)

<a id='msaflagging'></a>
## The `MSA Flagging` step

#### Summary

This is a NIRSpec-specific step, particularly for the MSA and IFU modes. The step flags pixels that are affected by MSA shutters that are stuck in the open position.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/msaflagopen/description.html) of the step.

#### Arguments

There are no optional arguments for this step.

#### Reference files used

This step uses the MSAOPER reference file, see the [reference_file_documentation](https://jwst-pipeline.readthedocs.io/en/stable/jwst/msaflagopen/reference_files.html) for details.


#### Run the step

In [ ]:
# Instantiate and set parameters
step = MSAFlagOpenStep()
step.output_dir = output_dir
step.save_results = True

# Call using the saturation instance from the previously-run step
result = step.run(result)

<a id='refpix_call'></a>
##### Using the call() method

In [ ]:
result = MSAFlagOpenStep.call(result, output_dir=output_dir,
                              save_results=True)

<a id='refpix_strun'></a>
##### From the command line

Here we show the commands that can be used from the command line.

<div class="alert alert-block alert-info">
    
```
strun jwst.msaflagopen.MSAFlagOpenStep previous_step.fits
````    
</div>

[Top of Notebook](#top)

<a id='extract_2d'></a>
## The `Extract 2D` step 

#### Summary

This step extracts 2D arrays from spectral images. The extractions are performed within all of the SCI, ERR, and DQ arrays of the input image model, as well as any variance arrays that may be present. 

It also computes an array of wavelengths to attach to the extracted data. The extracted arrays are stored as one or more slit objects in an output MultiSlitModel and saved as separate tuples of extensions in the output FITS file.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/extract_2d/main.html) of the step.

#### Arguments

This step has several [optional arguments](https://jwst-pipeline.readthedocs.io/en/stable/jwst/extract_2d/main.html#step-arguments)

#### Reference files used

This step uses the [`WAVELENGTHRANGE`](https://jwst-pipeline.readthedocs.io/en/stable/jwst/extract_2d/reference_files.html) reference file. 


#### Run the step

In [ ]:
# Instantiate and set parameters
step = Extract2dStep()
step.output_dir = output_dir
step.save_results = True

# Call using the saturation instance from the previously-run step
result = step.run(result)

[Top of Notebook](#top)

<a id='srctype'></a>
## The `Source Type` step 

#### Summary

This step ewstimates if a spectroscopic source should be considered to be a point or extended object, and populates the “SRCTYPE” main header keyword with a value of either “POINT” or “EXTENDED.” This information is then used in some subsequent spectroscopic processing steps to apply source-dependent corrections.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/srctype/description.html)

#### Arguments

There are no optional arguments for this step.

#### Reference files used

This step uses no specific reference files.


#### Run the step

In [ ]:
# Instantiate and set parameters
step = SourceTypeStep()
step.output_dir = output_dir
step.save_results = True

# Call using the saturation instance from the previously-run step
result = step.run(result)

<a id='masterbg'></a>
##  The `Master Background Subtraction` step 

#### Summary

This step subtracts background signal from 2-D spectroscopic data using a 1-D master background spectrum. 

The 1-D master background spectrum is created from one or more input exposures, or can alternatively be supplied by the user. 

The 1-D background spectrum - surface brightness versus wavelength - is projected into the 2-D space of source data based on the wavelength of each pixel in the 2-D data. The resulting 2-D background signal is then subtracted directly from the 2-D source data.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/master_background/description.html) of the step.

#### Arguments

This steps takes four optional arguments: user_background, save_background, force_subtract, output_use_model. For further details please see the [documentation](https://jwst-pipeline.readthedocs.io/en/stable/jwst/master_background/arguments.html)


#### Reference files used

This step uses no specific reference files.


#### Run the step

In [ ]:
# Instantiate and set parameters
step = MasterBackgroundStep()
step.output_dir = output_dir
step.save_results = True

# Call using the saturation instance from the previously-run step
result = step.run(result)

[Top of Notebook](#top)

<a id='wavecorr'></a>
## The `Wavecorr` step

#### Summary

This step is also NIRSpec specific, for FS and MOS modes. The wavelength correction (wavecorr) step updates the wavelength assignments for point sources that are known to be off center (in the dispersion direction) in their slit.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/wavecorr/description.html) of the step.

#### Arguments

This step has no step-specific arguments.

#### Reference files used

This step uses the [`WAVECORR`](https://jwst-pipeline.readthedocs.io/en/stable/jwst/wavecorr/reference_files.html).


#### Run the step

In [ ]:
# Instantiate and set parameters
step = WavecorrStep()
step.output_dir = output_dir
step.save_results = True

# Call using the saturation instance from the previously-run step
result = step.run(result)

[Top of Notebook](#top)

<a id='flat_field'></a>
## The `Flat Field` step

#### Summary

This step flat-fields an input science data set by dividing by a flat-field reference image. In particular, the SCI array from the flat-field reference file is divided into both the SCI and ERR arrays of the science data set, and the flat-field DQ array is combined with the science DQ array using a bitwise OR operation. 

Flat field correction for NIRSpec modes calculates an on-the-fly flat rather than reading data from CRDS. 


#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/flatfield/main.html) of the step.

#### Arguments

This step has only one optional argument, used for NIRSpec modes [save_interpolated_flat](https://jwst-pipeline.readthedocs.io/en/stable/jwst/flatfield/arguments.html).

#### Reference files used

This step uses several reference files [`Non-NIRSpec flats`](https://jwst-pipeline.readthedocs.io/en/stable/jwst/flatfield/reference_files.html) and [`NIRSpec flats`](https://jwst-pipeline.readthedocs.io/en/stable/jwst/flatfield/reference_files.html#reference-files-for-nirspec-spectroscopy).


#### Run the step

In [ ]:
# Instantiate and set parameters
step = FlatFieldStep()
step.output_dir = output_dir
step.save_results = True

# Call using the saturation instance from the previously-run step
result = step.run(result)

[Top of Notebook](#top)

<a id='straylight'></a>
## The `Straylight` step

#### Summary

This step is for MIRI MRS channel 1 and 2 data. The straylight correction uses information in the REGIONS reference file about which pixels belong to a slice and which pixels are located in the slice gaps.


#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/straylight/main.html) of the step.

#### Arguments

This step has three [optional arguments](https://jwst-pipeline.readthedocs.io/en/stable/jwst/straylight/arguments.html)


#### Reference files used

This step uses the [REGIONS](https://jwst-pipeline.readthedocs.io/en/stable/jwst/straylight/reference_files.html)


#### Run the step

In [ ]:
# Instantiate and set parameters
step = StraylightStep()
step.output_dir = output_dir
step.save_results = True

# Call using the saturation instance from the previously-run step
result = step.run(result)

[Top of Notebook](#top)

<a id='fringe'></a>
## The `Fringe` step

#### Summary




#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/flatfield/main.html) of the step.

#### Arguments



#### Reference files used




#### Run the step

In [ ]:
# Instantiate and set parameters
step = FringeStep()
step.output_dir = output_dir
step.save_results = True

# Call using the saturation instance from the previously-run step
result = step.run(result)

[Top of Notebook](#top)

<a id='pathloss'></a>
## The `Pathloss` step

#### Summary




#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/flatfield/main.html) of the step.

#### Arguments



#### Reference files used




#### Run the step

In [ ]:
# Instantiate and set parameters
step = PathLossStep()
step.output_dir = output_dir
step.save_results = True

# Call using the saturation instance from the previously-run step
result = step.run(result)

[Top of Notebook](#top)

<a id='barshadow'></a>
## The `Barshadow` step

#### Summary

This step is also NIRSpec specific, for MOS data. The step calculates the correction to be applied to NIRSpec MSA data for extended sources due to the bar that separates adjacent microshutters. This correction is applied to MultiSlit data after the pathloss correction has been applied.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/barshadow/description.html) of the step.

#### Arguments

This step has no specific arguments.

#### Reference files used

This step uses the [BARSHADOW reference file](https://jwst-pipeline.readthedocs.io/en/stable/jwst/barshadow/reference_files.html)


#### Run the step

In [ ]:
# Instantiate and set parameters
step = BarShadowStep()
step.output_dir = output_dir
step.save_results = True

# Call using the saturation instance from the previously-run step
result = step.run(result)

[Top of Notebook](#top)

<a id='photom'></a>
## The `Photom` step

#### Summary



#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/barshadow/description.html) of the step.

#### Arguments



#### Reference files used




#### Run the step

In [ ]:
# Instantiate and set parameters
step = PhotomStep()
step.output_dir = output_dir
step.save_results = True

# Call using the saturation instance from the previously-run step
result = step.run(result)

[Top of Notebook](#top)

<a id='resample'></a>
## The `Resample Spec` step

#### Summary

This step resamples each input 2D image based on the WCS and distortion information, and combines multiple resampled images into a single undistorted product. The distortion information should have been incorporated into the image using the assign_wcs step.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/main.html) of the step.

#### Arguments

This step has multiple [optional arguments](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/arguments.html)

#### Reference files used

This step uses the [DRIZPARS](https://jwst-pipeline.readthedocs.io/en/stable/jwst/resample/reference_files.html) reference file.


#### Run the step

In [ ]:
# Instantiate and set parameters
step = ResampleSpecStep()
step.output_dir = output_dir
step.save_results = True

# Call using the saturation instance from the previously-run step
result = step.run(result)

[Top of Notebook](#top)

<a id='cube_build'></a>
## The `Cube Build` step 

#### Summary

This step runs for MIRI or NIRSpec IFU calibrated 2-D images and produces 3-D spectral cubes. The 2-D disjointed IFU slice spectra are corrected for distortion and assembled into a rectangular cube with three orthogonal axes: two spatial and one spectral.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/cube_build/main.html) of the step.

#### Arguments

This step has multiple [optional arguments](https://jwst-pipeline.readthedocs.io/en/stable/jwst/cube_build/arguments.html)

#### Reference files used

This step uses the [CUBEPAR](https://jwst-pipeline.readthedocs.io/en/stable/jwst/cube_build/reference_files.html) reference file.


#### Run the step

In [ ]:
# Instantiate and set parameters
step = CubeBuildStep()
step.output_dir = output_dir
step.save_results = True

# Call using the saturation instance from the previously-run step
result = step.run(result)

[Top of Notebook](#top)

<a id='extract_1d'></a>
## The `Extract 1D` step 

#### Summary

This step extracts a 1D signal from a 2D or 3D dataset and writes spectral data to one of the calwebb_spec2 output files, “x1d”. This works on all JWST spectroscopic modes, including MIRI LRS (slit and slitless) and MRS, NIRCam WFSS and TSGRISM, NIRISS WFSS and SOSS, and NIRSpec fixed-slit, IFU, and MOS.

#### Documentation

[Full description](https://jwst-pipeline.readthedocs.io/en/stable/jwst/extract_1d/description.html) of the step.

#### Arguments

This step has multiple [optional arguments](https://jwst-pipeline.readthedocs.io/en/stable/jwst/extract_1d/arguments.html)

#### Reference files used

This step uses multiple [reference files](https://jwst-pipeline.readthedocs.io/en/stable/jwst/extract_1d/reference_files.html).


#### Run the step

In [ ]:
# Instantiate and set parameters
step = Extract1dStep()
step.output_dir = output_dir
step.save_results = True

# Call using the saturation instance from the previously-run step
result = step.run(result)

[Top of Notebook](#top)